### Exploratory Data Analysis

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [2]:
df = pd.read_csv("../data/1_work_data/processed/proc_2024-10-18.csv")

In [3]:
df.shape

(44884, 57)

In [4]:
missing_values = df.isnull().sum().sort_values(ascending=False)
print("\nMissing Values Count:\n", missing_values)


Missing Values Count:
 itr_yoy         11222
cr_yoy           5422
qr_yoy           5195
ev_eb_yoy        5118
ev_eb_qoq        4853
ev_ebitda        4799
adj_fperf        3831
fperf            3831
freturn          3831
pe_yoy           3387
vol_y            3386
price_yoy        3376
momentum_yoy     3376
rsi_60d          3171
pe_qoq           3167
vol_q            3161
price_qoq        3156
momentum_qoq     3156
rsi_30d          3148
rsi_14d          3136
vol_m            3126
momentum_mom     3123
price_mom        3123
pe               3116
ltcr_yoy         3094
ltda_yoy         2500
dlttq_yoy        2337
rtr_yoy          1744
dr_yoy            239
der_yoy           236
gpm_qoq           227
gpm_yoy           202
roe_yoy           182
atr_yoy           164
cfm               142
gpm               142
roe_qoq           113
roe               110
rev_yoy            97
roa_yoy            68
rev_qoq            62
ltq_yoy            52
ni_yoy             48
ni_2y              47
ltq_2y  

In [5]:
df['sector'].unique()

array(['Health Care', 'Financials', 'Industrials',
       'Consumer Discretionary', 'Information Technology',
       'Communication Services', 'Consumer Staples', 'Utilities',
       'Real Estate', 'Materials', 'Energy'], dtype=object)

In [6]:
sector_counts = df['sector'].value_counts()
fig_sector = px.bar(x=sector_counts.index, y=sector_counts.values, labels={'x': 'Sector', 'y': 'Count'}, title='Number of Stocks per Sector')
fig_sector.show()

In [7]:
sector_counts = df['fperf'].value_counts()
fig_sector = px.bar(x=sector_counts.index, y=sector_counts.values, title='Number of Over/Under performers')
fig_sector.show()

In [8]:
fig = px.histogram(df, x='fperf', marginal='box', title='Distribution of Future Returns')
fig.show()

### Single Stock Visualization

In [9]:
aapl_df = df[df.tic == 'AAPL']

In [10]:
fig = px.bar(aapl_df, x='tdq', y='roa')
fig.show()

In [11]:
fig = px.bar(aapl_df, x='rdq', y='roa_qoq')
fig.show()

### Histograms

In [12]:
fig_saleq = px.histogram(df, x='saleq', nbins=50, marginal='box', title='Distribution of Sales (saleq)')
fig_saleq.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['datadate', 'rdq', 'tdq', 'tic', 'sector', 'val_purch', 'val_sales', 'rsi_14d', 'rsi_30d', 'rsi_60d', 'vol_m', 'vol_q', 'vol_y', 'momentum_mom', 'momentum_qoq', 'momentum_yoy', 'price_mom', 'price_qoq', 'price_yoy', 'ni_qoq', 'ni_yoy', 'ni_2y', 'ltq_qoq', 'ltq_yoy', 'ltq_2y', 'dlttq_yoy', 'rev_qoq', 'rev_yoy', 'gpm', 'gpm_qoq', 'gpm_yoy', 'roa', 'roa_qoq', 'roa_yoy', 'roe', 'roe_qoq', 'roe_yoy', 'fcf_yoy', 'cfm', 'cr_yoy', 'qr_yoy', 'der_yoy', 'dr_yoy', 'ltda_yoy', 'pe', 'pe_qoq', 'pe_yoy', 'ev_ebitda', 'ev_eb_qoq', 'ev_eb_yoy', 'ltcr_yoy', 'itr_yoy', 'rtr_yoy', 'atr_yoy', 'freturn', 'fperf', 'adj_fperf'] but received: saleq